# CNN Transfer Learning with VGG16

In [10]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt 
%matplotlib inline

import keras
from keras.applications.vgg16 import preprocess_input 
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image

from keras.layers import Input, Flatten, Dense
from keras.models import Model

In [13]:
from keras import applications

input_img = tf.placeholder(tf.float32, shape=(None, 224, 224, 3))
print(input_img)

# model = VGG16(weights='imagenet', input_tensor=input_img)
model_vgg16 = VGG16(weights='imagenet', include_top=False)

Tensor("Placeholder_54:0", shape=(?, 224, 224, 3), dtype=float32)


In [14]:
model_vgg16.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [36]:
from keras.datasets import cifar10

num_classes = 10
batch_size = 100
epochs = 100

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [15]:
image_size = 32
channel = 3
input_layer = Input(shape=(image_size, image_size, channel), name='image_input')

In [18]:
# take the output from chopped-off vgg16 model 
output_model_vgg16 = model_vgg16(input_layer)

x = Flatten(name='flatten')(output_model_vgg16)
x = Dense(1024, activation='relu', name='fc1')(x)
x = Dense(1024, activation='relu', name='fc2')(x)
x = Dense(10, activation='softmax', name='predictions')(x)

new_model = Model(input=input_layer, output=x)
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 32, 32, 3)         0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
fc1 (Dense)                  (None, 1024)              525312    
_________________________________________________________________
fc2 (Dense)                  (None, 1024)              1049600   
_________________________________________________________________
predictions (Dense)          (None, 10)                10250     
Total params: 16,299,850
Trainable params: 16,299,850
Non-trainable params: 0
________________________________________________________________

/home/justin-y/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("im..., outputs=Tensor("pr...)`


In [19]:
layer_count = 0 
for layer in new_model.layers:
    layer_count = layer_count + 1
    print(layer)

In [20]:
# fix pretrained weights in vgg16 for training 
for i in range(layer_count-3):
    new_model.layers[i].trainable = False

## Train the new model 

In [21]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

new_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [22]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [35]:
new_model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, 
             validation_data=(x_test, y_test), shuffle=True)

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
 1800/50000 [>.............................] - ETA: 1038s - loss: 1.8288 - acc: 0.5389

KeyboardInterrupt: 